In [1]:
import joblib, pathlib, pandas as pd
model = joblib.load(pathlib.Path('../models/flare_xgb.pkl'))
print("Model OK:", model)

Model OK: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)


C:\Python313\Lib\pickle.py:1760: UserWarning: [10:42:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [5]:
import pandas as pd, numpy as np, joblib, pathlib
DATA = pathlib.Path('/content/drive/MyDrive/endoease_data')

# load cleaned sleep
sleep = pd.read_parquet(DATA/'sleep_clean.parquet')

# --- synthesize dummy labels for demo ---
sleep['flare_tomorrow'] = ((sleep['duration_h'] < 6) |
                          (sleep['quality_pct'] < 70)).shift(-1).fillna(0).astype(int)

# features / target
X = sleep[['duration_h', 'quality_pct']]
y = sleep['flare_tomorrow']

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

model = XGBClassifier(max_depth=4, n_estimators=100).fit(X_train, y_train)
pred = model.predict(X_test)
print("F1:", f1_score(y_test, pred))

# --- Create the directory inside Google Drive if it doesn't exist ---
# The parent directory of your data folder is '/content/drive/MyDrive'
model_dir = DATA.parent / 'models'
model_dir.mkdir(parents=True, exist_ok=True)

# --- Now you can safely dump the model to a persistent location ---
joblib.dump(model, model_dir/'flare_xgb.pkl')

F1: 1.0


['/content/drive/MyDrive/models/flare_xgb.pkl']